In [30]:
import pandas as pd

df = pd.read_csv('datos.csv', delimiter=';')

#print(df.head())

print(df.columns)

Index(['Categoría', 'Enlace', 'Nombre', 'ImagenURL', 'Calorías',
       'Ingredientes', 'Nutrientes', 'Vitaminas y Minerales'],
      dtype='object')


In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convertir la columna 'Ingredientes' en listas y aplicar One-Hot Encoding
df['Ingredientes'] = df['Ingredientes'].apply(lambda x: x.split(','))  # Convertir en listas
mlb = MultiLabelBinarizer()
ingredientes_encoded = mlb.fit_transform(df['Ingredientes'])

# Crear un DataFrame para los ingredientes codificados
ingredientes_df = pd.DataFrame(ingredientes_encoded, columns=mlb.classes_)
df = pd.concat([df, ingredientes_df], axis=1)

In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['Calorías'] = scaler.fit_transform(df[['Calorías']])


In [16]:
df['Categoría'] = df['Categoría'].astype('category')
df['Nutrientes'] = df['Nutrientes'].astype('category')


In [17]:
df['Baja_en_grasa'] = df['Nutrientes'].apply(lambda x: 1 if 'baja en grasa' in x else 0)
df['Alto_en_proteinas'] = df['Nutrientes'].apply(lambda x: 1 if 'alto en proteínas' in x else 0)


In [18]:
df['Es_desayuno'] = df['Categoría'].apply(lambda x: 1 if 'desayuno' in x.lower() else 0)
df['Es_almuerzo'] = df['Categoría'].apply(lambda x: 1 if 'almuerzo' in x.lower() else 0)
df['Es_cena'] = df['Categoría'].apply(lambda x: 1 if 'cena' in x.lower() else 0)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir los ingredientes en vectores numéricos
vectorizer = TfidfVectorizer()
ingredientes_vectores = vectorizer.fit_transform(df['Ingredientes'].apply(lambda x: ' '.join(x)))

# Normalizar las calorías y unir con los vectores de ingredientes
X = pd.concat([pd.DataFrame(ingredientes_vectores.toarray()), df[['Calorías']]], axis=1)


In [20]:
from sklearn.model_selection import train_test_split

# Definir las etiquetas de salida
y = df[['Baja_en_grasa', 'Alto_en_proteinas', 'Es_desayuno', 'Es_almuerzo', 'Es_cena']]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Crear el modelo
model = Sequential()

# Capa de entrada (tamaño igual a las características de entrada)
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))

# Capa oculta
model.add(Dense(64, activation='relu'))

# Capa de salida con activación sigmoide para clasificación binaria
model.add(Dense(y_train.shape[1], activation='sigmoid'))  # Una salida para cada etiqueta

# Compilación del modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


2025-02-24 17:41:19.660232: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 17:41:19.660793: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 17:41:19.663686: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 17:41:19.672707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740433279.687347   29740 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740433279.69

Epoch 1/10


/home/edfermachado/Descargas/Samsung/proyecto/ProyectoFinalSIC/miau/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-24 17:41:21.559013: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.3164 - loss: 0.6581 - val_accuracy: 0.0125 - val_loss: 0.4979
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0746 - loss: 0.4110 - val_accuracy: 0.0750 - val_loss: 0.2032
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1023 - loss: 0.1885 - val_accuracy: 0.0750 - val_loss: 0.1711
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1073 - loss: 0.1533 - val_accuracy: 0.0750 - val_loss: 0.1623
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1300 - loss: 0.1490 - val_accuracy: 0.0750 - val_loss: 0.1552
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1234 - loss: 0.1483 - val_accuracy: 0.0750 - val_loss: 0.1524
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1144 - loss: 0.1423 - val_accuracy: 0.0750 - val_loss: 0.1496
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1309 - loss: 0.1288 - val_accuracy: 0.0875 - val_loss: 0.1486
E

In [ ]:
import re
import nltk
import spacy
import os
from nltk.corpus import stopwords

# Obtener la ruta del directorio actual
directorio_actual = os.path.dirname(os.path.abspath(__file__))

# Descargar las stopwords de NLTK en el directorio actual
nltk.download('stopwords', download_dir="")

# Cargar el modelo de español de spaCy
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    # Descargar el modelo si no está instalado
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# Obtener las stopwords en español
stop_words = set(stopwords.words("spanish"))

def preprocesar_pregunta(pregunta):
    # Convertir la pregunta a minúsculas
    pregunta = pregunta.lower()

    # Eliminar caracteres especiales, números y puntuación
    pregunta = re.sub(r'[^a-záéíóúüñ\s]', '', pregunta)

    # Tokenizar la pregunta usando spaCy
    doc = nlp(pregunta)

    # Eliminar stopwords y aplicar lematización
    tokens = [token.lemma_ for token in doc if token.text not in stop_words]

    # Unir los tokens preprocesados de vuelta en una cadena
    pregunta_normalizada = " ".join(tokens)

    return pregunta_normalizada

# Ejemplo de uso
pregunta_ejemplo = "¿Cuáles son los mejores lugares para visitar en Madrid?"
pregunta_preprocesada = preprocesar_pregunta(pregunta_ejemplo)
print(pregunta_preprocesada)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edfermachado/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [29]:
def responder_a_consulta(pregunta):
    # Preprocesar la pregunta del usuario (convertir a minúsculas, tokenizar, etc.)
    pregunta_normalizada = preprocesar_pregunta(pregunta)

    # Convertir la pregunta en un vector utilizando TF-IDF
    pregunta_vectorizada = vectorizer.transform([pregunta_normalizada])

    # Predecir las etiquetas (ingredientes, características nutricionales, etc.)
    predicciones = model.predict(pregunta_vectorizada)

    # Mapear las predicciones a las recetas relevantes
    recetas = df[predicciones > 0.5]  # Ajusta el umbral según lo que necesites
    return recetas[['Nombre', 'Ingredientes']]




# Ejemplo de uso
pregunta = "Receta de desayuno con pollo"
resultados = responder_a_consulta(preprocesar_pregunta(pregunta))

# Mostrar los resultados
if isinstance(resultados, list):
    for receta in resultados:
        print(f"Receta: {receta['Nombre']}, Categoría: {receta['Categoría']}, Ingredientes: {receta['Ingredientes']}")
else:
    print(resultados)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 800, but received input with shape (1, 799)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 799), dtype=float32)
  • training=False
  • mask=None

In [32]:
import re
import nltk
import spacy
from nltk.corpus import stopwords
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.feature_extraction.text import TfidfVectorizer

# Asegurémonos de que el vectorizador esté entrenado con todos los datos de ingredientes
vectorizer = TfidfVectorizer()

# Entrenar el vectorizador con los ingredientes del DataFrame completo
ingredientes_vectores = vectorizer.fit_transform(df['Ingredientes'].apply(lambda x: ' '.join(x)))

# Comprobar el número de características del vectorizador
print(f"Número de características en el vectorizador: {ingredientes_vectores.shape[1]}")

# Luego, asegurarnos de que en la predicción se mantenga el mismo número de características
def preprocesar_pregunta(pregunta):
    # Convertir la pregunta a minúsculas
    pregunta = pregunta.lower()

    # Eliminar caracteres especiales, números y puntuación
    pregunta = re.sub(r'[^a-záéíóúüñ\s]', '', pregunta)

    # Tokenizar la pregunta usando spaCy
    doc = nlp(pregunta)

    # Eliminar stopwords y aplicar lematización
    tokens = [token.lemma_ for token in doc if token.text not in stop_words]

    # Unir los tokens preprocesados de vuelta en una cadena
    pregunta_normalizada = " ".join(tokens)

    return pregunta_normalizada

def responder_a_consulta(pregunta):
    # Preprocesar la pregunta
    pregunta_normalizada = preprocesar_pregunta(pregunta)

    # Convertir la pregunta en un vector utilizando el mismo TF-IDF que entrenamos
    pregunta_vectorizada = vectorizer.transform([pregunta_normalizada])

    # Verificar que el número de características de la entrada coincide con el modelo
    if pregunta_vectorizada.shape[1] != X_train.shape[1]:
        raise ValueError(f"El número de características es incompatible. Esperado {X_train.shape[1]}, pero se recibió {pregunta_vectorizada.shape[1]}.")

    # Predecir las probabilidades para cada etiqueta
    predicciones = model.predict(pregunta_vectorizada)

    # Mapear las predicciones
    etiquetas_predichas = (predicciones > 0.5).astype(int)  # Ajuste de umbral a 0.5 para todas las etiquetas

    # Filtrar las recetas que cumplen con las condiciones
    recetas_filtradas = df.copy()

    # Aplicar las etiquetas predichas
    if etiquetas_predichas[0][0] == 1:  # Baja_en_grasa
        recetas_filtradas = recetas_filtradas[recetas_filtradas['Baja_en_grasa'] == 1]
    if etiquetas_predichas[0][1] == 1:  # Alto_en_proteinas
        recetas_filtradas = recetas_filtradas[recetas_filtradas['Alto_en_proteinas'] == 1]
    if etiquetas_predichas[0][2] == 1:  # Es_desayuno
        recetas_filtradas = recetas_filtradas[recetas_filtradas['Es_desayuno'] == 1]
    if etiquetas_predichas[0][3] == 1:  # Es_almuerzo
        recetas_filtradas = recetas_filtradas[recetas_filtradas['Es_almuerzo'] == 1]
    if etiquetas_predichas[0][4] == 1:  # Es_cena
        recetas_filtradas = recetas_filtradas[recetas_filtradas['Es_cena'] == 1]

    # Mostrar resultados
    if not recetas_filtradas.empty:
        return recetas_filtradas[['Nombre', 'Ingredientes']]
    else:
        return "No se encontraron recetas que coincidan con los parámetros."

# Ejemplo de uso
pregunta = "Receta de desayuno con pollo"
resultados = responder_a_consulta(pregunta)

# Mostrar los resultados
if isinstance(resultados, pd.DataFrame):
    for _, receta in resultados.iterrows():
        print(f"Receta: {receta['Nombre']}, Ingredientes: {receta['Ingredientes']}")
else:
    print(resultados)



TypeError: can only join an iterable